In [2]:
import pandas as pd
import random

In [ ]:
prod_df = pd.read_csv('products_reduced_full_walkback_reverse.csv')
print(prod_df.shape)
print(prod_df.columns)

In [ ]:
filter_df = prod_df[prod_df.CategoryLevel3Name.notnull()].copy(deep=True)

id_cols = [col for col in filter_df.columns if 'Id' in col]
drop_cols = id_cols + ['CategoryLevel4Name','CategoryLevel5Name','CategoryLevel6Name','CategoryLevel7Name']
filter_df.drop(drop_cols, axis=1, inplace=True)
print(drop_cols)
print(filter_df.shape)

In [ ]:
filter_df.to_csv('products_filtered.csv', encoding='utf-8')

In [ ]:
del prod_df

In [8]:
filter_df = pd.read_csv('products_filtered.csv', index_col=0)
random.seed(a=210)

hier_cols = [col for col in filter_df.columns if 'CategoryLevel' in col]
train_cols = ['train_' + col for col in hier_cols]
print(hier_cols)

['CategoryLevel0Name', 'CategoryLevel1Name', 'CategoryLevel2Name', 'CategoryLevel3Name']


In [9]:
filter_df[train_cols] = filter_df[hier_cols].copy(deep=True)
filter_df.reset_index(inplace=True)
filter_df['train_status'] = 'Correct Record'

error_rate = .05
errors_per_col = round((error_rate / len(train_cols)) * filter_df.shape[0])
print('Errors per Column: {0} rows'.format(errors_per_col))

def pick_error(cur_val, all_values):
    sub_set = all_values.copy()
#     print(cur_val)
    sub_set.remove(cur_val)
#     print(sub_set)
    return(sub_set[random.randint(0, len(sub_set) - 1)])

def update_status(status, col):
    append_status = 'error: {0} |'.format(col)
    if status == 'Correct Record':
        return append_status
    else:
        return status + append_status

for i, col in enumerate(train_cols):
    error_rows = random.sample(range(filter_df.shape[0]), errors_per_col)
    all_values = filter_df[col].unique().tolist()
    filter_df[col].ix[error_rows] = filter_df[hier_cols[i]].ix[error_rows].apply(lambda x: pick_error(x, all_values))
    filter_df.train_status.ix[error_rows] = filter_df.train_status.ix[error_rows].apply(lambda x: update_status(x, col))

print(filter_df.train_status.value_counts())

Errors per Column: 12291 rows


C:\Users\timothy.weeks\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [13]:
filter_df.to_csv('master_products_5per_error.csv', encoding='utf-8')